In [93]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-120597")
exp = Experiment(workspace=ws, name="quick-starts-ws-120597")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-120597
Azure region: southcentralus
Subscription id: 8e713106-916f-4177-890e-435b90d7adc4
Resource group: aml-quickstarts-120597


In [94]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
try:
    cpu_cluster_compute_target = ComputeTarget(workspace=ws, name='cpu-compute-trn')

    print('Found existing cluster, use it.')

except ComputeTargetException:
  compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
  cpu_cluster_compute_target = ComputeTarget.create(ws, 'cpu-compute-trn', compute_config)

cpu_cluster_compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [12]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--max_iter": choice(10,20,30),
    "--C": choice(0.5, 1, 1.5)
    })

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training', 
                    compute_target=cpu_cluster_compute_target,
                    entry_script='train.py')


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

remote_run = exp.submit(hyperdrive_config)

RunDetails(remote_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
import joblib
# Get your best run and save the model from that run.

best_run = remote_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Best Params:',parameter_values)

Best Run Id:  HD_76f4bd01-a4aa-43a7-a8d5-4de93d30746d_1

 Accuracy: 0.91442097596504

 Best Params: ['--C', '1', '--max_iter', '30']


In [95]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
file_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=file_path)

In [81]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


In [96]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=cpu_cluster_compute_target,
    featurization= "auto")

In [97]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output=True)

Running on remote.


Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No definitions exist for environment AutoML-Non-Prod",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null
  },
  "correlation": {
    "operation": "829f6a04ba3a9e4ba26982a1e54a92d7",
    "request": "18b652d0f6fb9545"
  },
  "environment": "southcentralus",
  "location": "southcentralus",
  "time": "2020-10-13T23:53:21.0196113+00:00",
  "componentName": "environment-management"
}

In [29]:
# monitor the run
RunDetails(remote_run).show()

ConfigException: ConfigException:
	Message: Cancel operation is not supported for local runs. Local runs may be canceled by raising a keyboard interrupt (e.g. via. `Ctrl + C`)
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cancel operation is not supported for local runs. Local runs may be canceled by raising a keyboard interrupt (e.g. via. `Ctrl + C`)",
        "target": "cancel",
        "inner_error": {
            "code": "NotSupported"
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()

joblib.dump(value=fitted_model, filename="best-trained-model.pkl")